In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [8]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [2]:
EPOCHS = 40
BATCH_SIZE = 64

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

100.1%

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw


113.5%

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw


100.4%

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw


180.4%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw
Processing...


..\torch\csrc\utils\tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Done!


In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [9]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [10]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item())) 

In [11]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [12]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.265435
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.411680
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.454014
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.557403
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.587506
[1] Test Loss: 0.1826, Accuracy: 94.91%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.320172
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.309385
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.265017
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.350152
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.194963
[2] Test Loss: 0.1167, Accuracy: 96.35%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.379598
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.311678
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.353995
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.516980
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.237431
[3] Test Loss: 0.0957, Accuracy: 97.03%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.277666
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.240060
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.177865


Train Epoch: 29 [38400/60000 (64%)]	Loss: 0.279338
Train Epoch: 29 [51200/60000 (85%)]	Loss: 0.249331
[29] Test Loss: 0.0327, Accuracy: 98.89%
Train Epoch: 30 [0/60000 (0%)]	Loss: 0.095009
Train Epoch: 30 [12800/60000 (21%)]	Loss: 0.105525
Train Epoch: 30 [25600/60000 (43%)]	Loss: 0.072435
Train Epoch: 30 [38400/60000 (64%)]	Loss: 0.015196
Train Epoch: 30 [51200/60000 (85%)]	Loss: 0.254269
[30] Test Loss: 0.0357, Accuracy: 98.85%
Train Epoch: 31 [0/60000 (0%)]	Loss: 0.156988
Train Epoch: 31 [12800/60000 (21%)]	Loss: 0.156511
Train Epoch: 31 [25600/60000 (43%)]	Loss: 0.394644
Train Epoch: 31 [38400/60000 (64%)]	Loss: 0.100256
Train Epoch: 31 [51200/60000 (85%)]	Loss: 0.181751
[31] Test Loss: 0.0353, Accuracy: 98.84%
Train Epoch: 32 [0/60000 (0%)]	Loss: 0.117506
Train Epoch: 32 [12800/60000 (21%)]	Loss: 0.094931
Train Epoch: 32 [25600/60000 (43%)]	Loss: 0.160781
Train Epoch: 32 [38400/60000 (64%)]	Loss: 0.056352
Train Epoch: 32 [51200/60000 (85%)]	Loss: 0.094177
[32] Test Loss: 0.0341, A